In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

## Word2Vec(Skipgram, CBOW) TensorFlow로 구현

### Skipgram dataset 만들기

In [ ]:
# train_dataset
def generate_skipgram_dataset(original_data, window_size, batch_size):
    """
    skipgram model에 들어갈 수 있는 데이터셋 생성.

    parameters
    ----------
    original_data: list of strings. 데이터셋으로 바꿔야 할, 리스트로 이루어진 텍스트 데이터.
    window_size: int. sliding window 크기
    batch_size: int.

    return
    ------
    dataset: model.fit에 바로 넣을 수 있는 데이터셋.
    """
    # 단어 리스트
    words = []
    for sentence in original_data:
        for word in sentence.split():
            words.append(word)
    words = list(set(words))

    # 단어를 index에 mapping
    word2index = {w:i for i, w in enumerate(words)}
    index2word = {i:w for i, w in enumerate(words)}

    # skipgram data 만들기
    pairs = []
    for sentence in original_data:
        tokens = sentence.split()
        for idx, token in enumerate(tokens):
            start = max(idx-window_size, 0)
            end = min(idx+window_size+1, len(tokens))
            for neighbor in tokens[start:end]:
                if neighbor != token:
                    pairs.append((word2index[token], word2index[neighbor]))

    # dataset으로 변환
    dataset = tf.data.Dataset.from_tensor_slices((pairs[:, 0], pairs[:, 1]))
    dataset = dataset.shuffle(len(pairs)).batch(batch_size, drop_remainder=True)

    return dataset


In [2]:
# wikipedia dump 다운로드
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2

--2024-03-15 09:40:17--  https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21869935187 (20G) [application/octet-stream]
Saving to: ‘enwiki-latest-pages-articles.xml.bz2’

es.xml.bz2            0%[                    ]  35.94M  4.70MB/s    eta 71m 43s^C


In [ ]:
import bz2
from bs4 import BeautifulSoup

def extract_text_from_bz2_dump(file_path):
    texts = []  # 추출된 텍스트를 저장할 리스트

    with bz2.open(file_path, 'rb') as file:
        for line in file:
            # 각 줄을 파싱하기 위해 BeautifulSoup 사용
            soup = BeautifulSoup(line, 'lxml')

            # Wikipedia 문서의 텍스트 내용이 <text> 태그 안에 있음
            for doc in soup.find_all('text'):
                text = doc.get_text()
                texts.append(text)

    return texts

# 함수 사용 예시
file_path = 'enwiki-latest-pages-articles.xml.bz2'  # 다운로드한 파일의 경로
texts = extract_text_from_bz2_dump(file_path)

# 처음 5개의 문서 텍스트 출력
for text in texts[:5]:
    print(text[:100])  # 각 문서의 처음 100자만 출력


In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=False)

train_data = dataset['train']
train_text = train_data.map(lambda x:x['text'])
train_text = train_text.numpy().decode('utf-8').lower()

corpus = []
for text in train_text:
    words = text.split()
    corpus.append(' '.join(words))

train_dataset = generate_skipgram_dataset(corpus, window_size=5, batch_size=32)

### Skipgram
skigram은 중심단어(target)로부터 주변단어(context)를 예측하는 구조로, 이를 통해 embedding matrix를 학습합니다.  
  
구현하는 방법은 주로 중심단어와 주변단어의 내적(dot product)을 계산하여 유사도를 구하는 것입니다. (코사인 특성상) 두 벡터가 유사할수록 내적의 값이 크고, 유사하지 않을수록 내적의 값이 작습니다. 이 내적값을 그대로 로짓으로 삼아 소프트맥스와 같은 함수에게 주어 각 클래스별 확률값을 예측하고, categorical crossentropy 손실함수를 사용하여 실제 확률분포와의 차이를 최소화합니다. 이를 통해 올바른 클래스를 예측할 수 있는 embedding matrix를 학습하게 됩니다.

In [ ]:
class SkipGramModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        self.target_embedding = tf.keras.layers.Embedding(vocab_size,
                                                          embedding_dim,
                                                          input_length=1)
        self.context_embedding = tf.keras.layers.Embedding(vocab_size,
                                                           embedding_dim,
                                                           input_length=1)
        self.dots = tf.keras.layers.Dot(axis=-1)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, pair):
        target, context = pair
        target_embedding = self.target_embedding(target)
        context_embedding = self.context_embedding(context)
        dots = self.dots([target_embedding, context_embedding])
        flatten = self.flatten(dots)
        return flatten

In [ ]:
vocab_size = 10000
embedding_dim = 300

skipgram_model = SkipGramModel(vocab_size, embedding_dim)

skipgram_model.compile(
    optimizer='adam',
    loss = tf.keras.losses.CategoricalCrossEntropy(from_logits=True),
    metrics = ['accuracy']
)

skipgram_dataset = generate_skipgram_dataset(data, vocab_size, embedding_dim)

skipgram_model.fit(skipgram_dataset, epochs=10)
s_history = skipgram_model.history()

# 중심단어의 embedding matrix
skipgram_embedding_matrix = skipgram_model.target_embedding.get_weights()[0]

### CBOW
cbow는 주변단어(context)로부터 중심단어(target)를 예측하는 구조입니다.

In [ ]:
# cbow dataset
def generate_cbow_dataset(original_data, window_size, batch_size):
    """
    cbow model에 들어갈 수 있는 데이터셋 생성.

    parameters
    ----------
    original_data: list of strings. 데이터셋으로 바꿔야 할, 리스트로 이루어진 텍스트 데이터.
    window_size: int. sliding window 크기
    batch_size: int.

    return
    ------
    dataset: model.fit에 바로 넣을 수 있는 데이터셋.
    """
    # 단어 리스트
    words = []
    for sentence in original_data:
        for word in sentence.split():
            words.append(word)
    words = list(set(words))

    # 단어를 index에 mapping
    word2index = {w:i for i, w in enumerate(words)}
    index2word = {i:w for i, w in enumerate(words)}

    # cbow data 만들기
    data = []
    for sentence in original_data:
        tokens = [word2index[word] for word in sentence.split()]
        for idx, token in enumerate(tokens):
            start = max(idx-window_size, 0)
            end = min(idx+window_size+1, len(tokens))
            context = [neighbor for neighbor in tokens[start:end] if neighbor != token]
            if len(context)>0:
                data.append((context, token))

    contexts, targets = zip(*data)
    contexts = tf.keras.preprocessing.sequence.pad_sequences(contexts, padding='post')
    targets = np.array(targets)

    dataset = tf.data.Dataset.from_tensor_slices((contexts, targets))
    dataset = dataset.shuffle(len(targets)).batch(batch_size, drop_remainder=True)

    return dataset


In [ ]:
# CBOW 모델 정의
class CBOWModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOWModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size,
                                                   embedding_dim,
                                                   input_length=None)
        self.global_average_pooling = tf.keras.layers.GlobalAveragePooling1D()
        self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax')

    def call(self, context):
        context_embedding = self.embedding(context)
        pooled_context = self.global_average_pooling(context_embedding)
        predictions = self.dense(pooled_context)
        return predictions

In [ ]:

cbow_model = CBOWModel(vocab_size, embedding_dim)

cbow_model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

cbow_dataset = generate_cbow_dataset(data, vocab_size, embedding_dim)

cbow_model.fit(cbow_dataset, epochs=10)
c_history = cbow_model.history()

# 주변단어(context)의 embedding matrix
cbow_embedding_matrix = cbow_model.embedding.get_weights()[0]

### skipgram & cbow by word2vec function

### Comparison via visualization (tensorflow vs word2vec)